# 7. 의사결정트리 실습

In [39]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# noise: 잡음의 크기. 0이면 정확한 반원을 이룸
X, y = make_moons(n_samples=1000, noise=0.4, random_state=42)

In [40]:
X[:5] # 좌표인 것 같음

array([[-0.23206028,  0.67227396],
       [ 1.36391564, -0.18824971],
       [ 0.65089014,  0.47532731],
       [-0.24222196, -0.75845734],
       [-0.78252032,  0.56187483]])

In [41]:
y[:5] # 0,1 클래스인 것 같음

array([1, 1, 1, 1, 0])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [55]:
# DecisionTreeClassifierd 최적화
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# n_jobs = -1로 설정 하시면 모든 코어사용이 가능합니다. Default값은 1 입니다.
# cv = 교차검증을 위한 fold 횟수 입니다. KFold를 쓰고 싶으신 분은 KFold를 설정해주셔도 됩니다.
# Ex) cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42)
# refit : True면 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 해당 하이퍼 파라미터로 재학습시킨다는 의미 입니다 . Default 값은 True입니다.
# scoring : 찾으시는 다른 값으로 변경 가능합니다.

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=1)

In [56]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=7, random_state=42)

In [65]:
from sklearn.model_selection import cross_val_score

score =cross_val_score(grid_search_cv.best_estimator_,X_test,y_test, scoring='accuracy')
print(score)

[0.93333333 0.83333333 0.85       0.86666667 0.83333333]


# 8. 랜덤포레스트 만들기

In [58]:
# 1. 훈련세트의 100개 무작위 샘플로 이루어진 서브셋 1000개 생성
# shufflesplit 함수는 데이터셋 인덱스를 무작위로 사전에 설정한 비율로 분할합니다. 
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [62]:
# 2. 각 테스트 세트 샘플에 대해 1,000개의 결정 트리 예측을 만들고 다수로 나온 예측만 취합니다
from sklearn.base import clone
from sklearn.metrics import accuracy_score

forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)

In [63]:
# 3. 다수로 나온 예측만 취합니다
from scipy.stats import mode
import numpy as np

Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

/var/folders/41/615730cj21lfyfd4hqhcdcbr0000gn/T/ipykernel_71609/2270376661.py:8: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)


In [64]:
#앞서 만든 모델보다 조금 높은(약 0.5~1.5% 정도) 정확도를 얻게 될 것입니다. 
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8633333333333333